In [1]:
# !python /home/vs428/Documents/deep-ed-diags/simple_nn_runner.py

# Test on Data

In [150]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [151]:
# Path hack.
import sys
import os
sys.path.append(os.path.abspath('..'))

In [152]:
import csv
import ast
from datetime import date, datetime, time, timedelta
from datetime import datetime
import pickle as pkl
import time

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.utils.class_weight import compute_class_weight    

import numpy as np
import pandas as pd
import sklearn
from joblib import dump, load

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from models.simple_nn import *

In [153]:
# N_CLASSES = 65
# model = AbdPainPredictionMLP(N_CLASSES)
# # Model class must be defined somewhere
# model = torch.load("/gpfs/milgram/project/rtaylor/shared/ABDPain_EarlyDiags/models/frosty-spaceship-51.model")
model = torch.load("/gpfs/milgram/project/rtaylor/shared/ABDPain_EarlyDiags/models/cute-quiver-145.model")
# model = torch.load("/gpfs/milgram/project/rtaylor/shared/ABDPain_EarlyDiags/models/prime-water-50.model")
model.eval()

AbdPainPredictionMLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=1421, out_features=512, bias=True)
    (1): Dropout(p=0.0, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): Dropout(p=0.0, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): Dropout(p=0.0, inplace=False)
    (8): ReLU()
    (9): Linear(in_features=256, out_features=58, bias=True)
  )
)

In [154]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# Dataset

In [155]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
# from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold


import optuna


import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import ConfusionMatrixDisplay

In [157]:
data = pd.read_csv(f"/gpfs/milgram/project/rtaylor/shared/ABDPain_EarlyDiags/unq_pt_enc_single_label_full_clean_label.csv",
#                   nrows=50000
                  )


In [1]:

print(f"dataset size: {data.shape}")

single_support_classes = set(data['label'].value_counts()[data['label'].value_counts() == 1].index)
# droppable_rows = data['label'].isin(single_support_classes).sum()
data = data[~data['label'].isin(single_support_classes)]

print(f"dataset size: {data.shape}")


NameError: name 'data' is not defined

In [160]:
data['label'].value_counts()

Abdominal Pain, general                68321
Acute cystitis                         13695
Urolithiasis                           11556
Colitis                                 9204
Constipation                            8679
                                       ...  
Labor/Preterm Labor                        6
Porphyria                                  5
Uremia                                     4
Paroxysmal nocturnal hemoglobinuria        3
Placental Abruption                        2
Name: label, Length: 64, dtype: int64

In [169]:
# data
# class_weights = pd.Series(compute_class_weight(class_weight="balanced",classes=samples_per_cls.index, y=data['label']),
#                     index=samples_per_cls.index )
x1 = compute_class_weight(class_weight="balanced",classes=data['label'].value_counts().index, y=data['label'])
samples_per_cls = data['label'].value_counts()
x2 = data.shape[0]/(len(samples_per_cls) * samples_per_cls)
# compute_class_weight(data)

In [ ]:
DOWNSAMPLE = True
if DOWNSAMPLE:
    # we use the avg of the freq of the next 5 classes to downsample abdominal pain
    downsample_rate = int(data['label'].value_counts()[1:5].mean())
    def downsample_grp(grp):
        if grp.name == "Abdominal Pain, general":
            return grp.sample(downsample_rate)
        else:
            return grp
    downsampled = data.groupby("label").apply(downsample_grp)    
    data = downsampled.drop("label", axis=1).reset_index().set_index("level_1")
    data.index.name = None
    # reorder the cols to put the 'label' col back to the end
    cols = data.columns.tolist()
    cols = cols[1:] + [cols[0]]
    data = data[cols]
    
    train_test_stratify = data['label']

else:
    train_test_stratify = data['label']


In [138]:

# drop all columns that don't have any positive actual values/only have all NaNs
data = data.drop( data.columns[((data.shape[0]- data.isnull().sum()) == 0)], 
                 axis=1, errors='ignore' )
print(f"After dropping sparse columns: {data.shape}")


After dropping sparse columns: (123232, 2275)


In [139]:

# remove columns that don't have at least N (hyperparam) number of non-NaN values
data = data[ data.columns.intersection(data.columns[(((data.shape[0]- data.isnull().sum())) > 5)]) ]
print(f"After dropping sparse columns: {data.shape}")

data.columns = data.columns.str.replace("[|]|<", "leq_")


After dropping sparse columns: (123232, 2274)


/gpfs/milgram/project/rtaylor/vs428/conda_envs/factcheck_env/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [140]:
data

,PAT_ENC_CSN_ID,person_id,visit_occurrence_id,age,EdDisposition_AMA,EdDisposition_Admit,EdDisposition_Discharge,EdDisposition_Eloped,EdDisposition_Expired,EdDisposition_LWBS after Triage,...,last_BP_Diastolic,ed_SpO2,ed_Temp,ed_Patient Acuity,ed_Pulse,ed_Pain Score,ed_Resp,ed_BP_Systolic,ed_BP_Diastolic,label
41604,135059007,17260779,54537564,23.0,NaN,NaN,1.0,NaN,NaN,NaN,...,82.0,95.0,98.3,3.0,63.0,6.0,18.0,129.0,84.0,"Abdominal Pain, general"
75500,165959607,13482905,92226980,70.0,NaN,NaN,1.0,NaN,NaN,NaN,...,83.0,96.0,98.5,4.0,74.0,6.0,16.0,135.0,89.0,"Abdominal Pain, general"
68109,159837663,11300331,85092268,17.0,NaN,NaN,1.0,NaN,NaN,NaN,...,64.0,99.0,99.0,3.0,84.0,8.0,18.0,116.0,65.0,"Abdominal Pain, general"
3857,96814996,10661884,17755109,50.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,100.0,98.7,3.0,77.0,8.0,18.0,126.0,80.0,"Abdominal Pain, general"
64320,156606751,19458030,80801002,40.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,98.0,98.4,3.0,75.0,10.0,18.0,148.0,103.0,"Abdominal Pain, general"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180137,249295488,43410437,239176261,3.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,97.0,98.4,3.0,125.0,0.0,22.0,82.0,52.0,Viral infection/influenza
180284,249461005,39415415,239394749,9.0,NaN,NaN,1.0,NaN,NaN,NaN,...,66.0,99.0,99.0,4.0,115.0,2.0,20.0,94.0,62.0,Viral infection/influenza
180892,249897275,35248045,239903069,58.0,NaN,NaN,1.0,NaN,NaN,NaN,...,56.0,98.0,98.3,3.0,85.0,8.0,17.0,108.0,79.0,Viral infection/influenza
181272,250235901,13725268,240619524,8.0,NaN,NaN,1.0,NaN,NaN,NaN,...,71.0,96.0,98.3,3.0,150.0,0.0,20.0,NaN,NaN,Viral infection/influenza


In [141]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

le.fit(data['label'])


LabelEncoder()

In [142]:
non_train_col_mask = data.columns[data.columns.str.contains("EdDisposition_")].union(data.columns[:], sort=False).union(pd.Index(['label']), sort=False)
train_col_mask = data.columns.difference(non_train_col_mask, sort=False)


In [143]:
# X_test, y_test = data[train_col_mask], data['label']
# Create a dummy index variable to get the indices
indices = range(data.shape[0])
X_train, X_test, y_train, y_test, idxs_train, idxs_test = train_test_split(data[train_col_mask],
                                                    data['label'],
                                                    indices,
                                                    stratify=train_test_stratify, 
                                                   test_size=0.2, random_state=314, 
                                                   )


In [144]:
X_test.shape

(24647, 0)

In [145]:
std_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

In [146]:

def flatten(t):
    return [item for sublist in t for item in sublist]


def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]



In [147]:
cat_cols = pd.Index(['DepartmentName_BH EMERGENCY DEPARTMENT', 'DepartmentName_BH FAIRFIELD URGENT CARE', 'DepartmentName_GH EMERGENCY DEPARTMENT', 'DepartmentName_LMH EMERGENCY DEPARTMENT', 'DepartmentName_SRC EMERGENCY DEPARTMENT', 'DepartmentName_WH EMERGENCY DEPARTMENT', 'DepartmentName_YHC ACUTE AND INPATIENT CARE', 'DepartmentName_YNH EMERGENCY ADULT', 'DepartmentName_YNH EMERGENCY PEDIATRIC', 'DepartmentName_YNH EMERGENCY SHORELINE', 'Sex_Female', 'Sex_Male', 'GenderIdentity_Choose not to disclose', 'GenderIdentity_Female', 'GenderIdentity_Gender non-conforming', 'GenderIdentity_Male', 'GenderIdentity_Transgender Female / Male-to-Female', 'GenderIdentity_Transgender Male / Female-to-Male', 'FirstRace_Asian', 'FirstRace_Black or African American', 'FirstRace_Native Hawaiian or Other Pacific Islander', 'FirstRace_Other', 'FirstRace_Other Pacific Islander', 'FirstRace_Other/Not Listed', 'FirstRace_Patient Refused', 'FirstRace_Unknown', 'FirstRace_White or Caucasian', 'Ethnicity_Hispanic or Latino', 'Ethnicity_Non-Hispanic', 'Ethnicity_Patient Refused', 'Ethnicity_Unknown', 'PreferredLanguage_Albanian', 'PreferredLanguage_Arabic', 'PreferredLanguage_Bengali', 'PreferredLanguage_Chinese (Mandarin)', 'PreferredLanguage_English', 'PreferredLanguage_French', 'PreferredLanguage_Haitian Creole', 'PreferredLanguage_Italian', 'PreferredLanguage_Korean', 'PreferredLanguage_Polish', 'PreferredLanguage_Portuguese', 'PreferredLanguage_Russian', 'PreferredLanguage_Sign Language', 'PreferredLanguage_Spanish', 'PreferredLanguage_TAIL_OTHER', 'PreferredLanguage_Turkish', 'PreferredLanguage_Unknown', 'PreferredLanguage_Urdu Pakistan', 'PreferredLanguage_Vietnamese', 'SmokingStatus_*Unspecified', 'SmokingStatus_Current Every Day Smoker', 'SmokingStatus_Current Some Day Smoker', 'SmokingStatus_Former Smoker', 'SmokingStatus_Heavy Tobacco Smoker', 'SmokingStatus_Light Tobacco Smoker', 'SmokingStatus_Never Assessed', 'SmokingStatus_Never Smoker', 'SmokingStatus_Never Smoker ', 'SmokingStatus_Passive Smoke Exposure - Never Smoker', 'SmokingStatus_Smoker, Current Status Unknown', 'SmokingStatus_Unknown If Ever Smoked', 'AcuityLevel_Emergent', 'AcuityLevel_Immediate', 'AcuityLevel_Less Urgent', 'AcuityLevel_Non-Urgent', 'AcuityLevel_Urgent', 'FinancialClass_BCBS', 'FinancialClass_Commercial', 'FinancialClass_Managed Care', 'FinancialClass_Medicaid', 'FinancialClass_Medicaid Managed Care', 'FinancialClass_Medicare', 'FinancialClass_Medicare Managed Care', 'FinancialClass_Other', 'FinancialClass_Self-pay', "FinancialClass_Worker's Comp", 'CC_ABDOMINAL DISTENTION', 'CC_ABDOMINAL PAIN', 'CC_ABDOMINAL PAIN PREGNANT', 'CC_EPIGASTRIC PAIN', 'CC_FLANK PAIN', 'CC_PELVIC PAIN', 'CC_SIDE PAIN'])
normal_cols = pd.Index(['age', 'ed_6690-2_avg', 'ed_1968-7_avg', 'ed_1916-6_avg', 'ed_1798-8_avg', 'ed_17861-6_avg', 'ed_30089-7_avg', 'ed_2132-9_avg', 'ed_1759-0_avg', 'ed_2692-2_avg', 'ed_3298-7_avg', 'ed_33959-8_avg', 'ed_5902-2_avg', 'ed_770-8_avg', 'ed_61152-5_avg', 'ed_45194-8_avg', 'ed_3040-3_avg', 'ed_18182-6_avg', 'ed_42757-5_avg', 'ed_4544-3_avg', 'ed_751-8_avg', 'ed_50560-2_avg', 'ed_16362-6_avg', 'ed_736-9_avg', 'ed_2519-7_avg', 'ed_2518-9_avg', 'ed_53115-2_avg', 'ed_6598-7_avg', 'ed_20405-7_avg', 'ed_2160-0_avg', 'ed_6768-6_avg', 'ed_711-2_avg', 'ed_742-7_avg', 'ed_33762-6_avg', 'ed_10839-9_avg', 'ed_704-7_avg', 'ed_71695-1_avg', 'ed_771-6_avg', 'ed_10466-1_avg', 'ed_706-2_avg', 'ed_731-0_avg', 'ed_32623-1_avg', 'ed_9830-1_avg', 'ed_2885-2_avg', 'ed_713-8_avg', 'ed_5905-5_avg', 'ed_48065-7_avg', 'ed_30341-2_avg', 'ed_3051-0_avg', 'ed_50562-8_avg', 'past_21394-2_avg', 'past_6690-2_avg', 'past_1968-7_avg', 'past_1916-6_avg', 'past_1798-8_avg', 'past_17861-6_avg', 'past_2132-9_avg', 'past_1759-0_avg', 'past_54218-3_avg', 'past_30471-7_avg', 'past_3298-7_avg', 'past_46130-1_avg', 'past_5902-2_avg', 'past_770-8_avg', 'past_61152-5_avg', 'past_45194-8_avg', 'past_3040-3_avg', 'past_18182-6_avg', 'past_4485-9_avg', 'past_42757-5_avg', 'past_4544-3_avg', 'past_751-8_avg', 'past_50560-2_avg', 'past_16362-6_avg', 'past_2276-4_avg', 'past_2614-6_avg', 'past_4498-2_avg', 'past_736-9_avg', 'past_2519-7_avg', 'past_53115-2_avg', 'past_6598-7_avg', 'past_20405-7_avg', 'past_2160-0_avg', 'past_6768-6_avg', 'past_711-2_avg', 'past_742-7_avg', 'past_33762-6_avg', 'past_10839-9_avg', 'past_704-7_avg', 'past_71695-1_avg', 'past_771-6_avg', 'past_10466-1_avg', 'past_14959-1_avg', 'past_14957-5_avg', 'past_706-2_avg', 'past_731-0_avg', 'past_42244-4_avg', 'past_32623-1_avg', 'past_2458-8_avg', 'past_9830-1_avg', 'past_2885-2_avg', 'past_713-8_avg', 'past_5905-5_avg', 'past_48065-7_avg', 'past_30341-2_avg', 'past_3051-0_avg', 'past_50562-8_avg', 'last_SpO2', 'last_Temp', 'last_Patient Acuity', 'last_Pulse', 'last_Pain Score', 'last_Resp', 'last_BP_Systolic', 'last_BP_Diastolic', 'ed_SpO2', 'ed_Temp', 'ed_Patient Acuity', 'ed_Pulse', 'ed_Pain Score', 'ed_Resp', 'ed_BP_Systolic', 'ed_BP_Diastolic'])
all_other_cols = pd.Index(['visit_occurrence_id', 'BLD001', 'BLD002', 'BLD003', 'BLD004', 'BLD005', 'BLD006', 'BLD007', 'BLD008', 'BLD010', 'CIR001', 'CIR002', 'CIR003', 'CIR004', 'CIR005', 'CIR006', 'CIR007', 'CIR008', 'CIR009', 'CIR011', 'CIR012', 'CIR013', 'CIR014', 'CIR015', 'CIR016', 'CIR017', 'CIR018', 'CIR019', 'CIR020', 'CIR021', 'CIR023', 'CIR024', 'CIR025', 'CIR026', 'CIR027', 'CIR028', 'CIR029', 'CIR030', 'CIR031', 'CIR032', 'CIR033', 'CIR034', 'CIR035', 'CIR036', 'CIR039', 'DIG001', 'DIG002', 'DIG003', 'DIG004', 'DIG005', 'DIG006', 'DIG007', 'DIG008', 'DIG009', 'DIG010', 'DIG011', 'DIG012', 'DIG013', 'DIG014', 'DIG015', 'DIG016', 'DIG017', 'DIG018', 'DIG019', 'DIG020', 'DIG021', 'DIG022', 'DIG023', 'DIG024', 'DIG025', 'EAR001', 'EAR002', 'EAR003', 'EAR004', 'EAR006', 'END001', 'END002', 'END003', 'END007', 'END008', 'END009', 'END010', 'END011', 'END012', 'END013', 'END014', 'END015', 'END016', 'EXT001', 'EXT003', 'EXT005', 'EXT007', 'EXT012', 'EXT016', 'EXT018', 'EXT019', 'EXT025', 'EXT026', 'EXT027', 'EYE001', 'EYE002', 'EYE003', 'EYE004', 'EYE005', 'EYE006', 'EYE007', 'EYE008', 'EYE009', 'EYE010', 'EYE012', 'FAC001', 'FAC003', 'FAC004', 'FAC005', 'FAC006', 'FAC007', 'FAC008', 'FAC009', 'FAC010', 'FAC012', 'FAC013', 'FAC014', 'FAC015', 'FAC016', 'FAC017', 'FAC019', 'FAC020', 'FAC021', 'FAC022', 'FAC023', 'FAC024', 'FAC025', 'GEN001', 'GEN002', 'GEN003', 'GEN004', 'GEN005', 'GEN006', 'GEN007', 'GEN008', 'GEN009', 'GEN010', 'GEN011', 'GEN012', 'GEN013', 'GEN014', 'GEN015', 'GEN016', 'GEN017', 'GEN018', 'GEN019', 'GEN020', 'GEN021', 'GEN022', 'GEN023', 'GEN024', 'GEN025', 'GEN026', 'INF001', 'INF002', 'INF003', 'INF004', 'INF005', 'INF006', 'INF007', 'INF008', 'INF009', 'INF010', 'INJ001', 'INJ002', 'INJ003', 'INJ004', 'INJ005', 'INJ006', 'INJ007', 'INJ008', 'INJ010', 'INJ011', 'INJ012', 'INJ013', 'INJ014', 'INJ017', 'INJ019', 'INJ020', 'INJ021', 'INJ022', 'INJ023', 'INJ024', 'INJ025', 'INJ026', 'INJ027', 'INJ028', 'INJ030', 'INJ031', 'INJ032', 'INJ033', 'INJ034', 'INJ035', 'INJ036', 'INJ037', 'INJ039', 'INJ040', 'INJ041', 'INJ042', 'INJ061', 'INJ073', 'MAL001', 'MAL002', 'MAL003', 'MAL004', 'MAL005', 'MAL007', 'MAL008', 'MAL009', 'MAL010', 'MBD001', 'MBD002', 'MBD003', 'MBD004', 'MBD005', 'MBD006', 'MBD007', 'MBD008', 'MBD009', 'MBD010', 'MBD011', 'MBD012', 'MBD013', 'MBD014', 'MBD017', 'MBD018', 'MBD019', 'MBD020', 'MBD021', 'MBD022', 'MBD024', 'MBD025', 'MBD026', 'MUS001', 'MUS002', 'MUS003', 'MUS004', 'MUS006', 'MUS007', 'MUS008', 'MUS009', 'MUS010', 'MUS011', 'MUS012', 'MUS013', 'MUS014', 'MUS016', 'MUS021', 'MUS022', 'MUS023', 'MUS024', 'MUS025', 'MUS026', 'MUS028', 'MUS029', 'MUS030', 'MUS031', 'MUS032', 'MUS033', 'MUS034', 'MUS037', 'MUS038', 'NEO002', 'NEO007', 'NEO008', 'NEO009', 'NEO010', 'NEO012', 'NEO013', 'NEO014', 'NEO015', 'NEO016', 'NEO017', 'NEO018', 'NEO020', 'NEO021', 'NEO022', 'NEO023', 'NEO024', 'NEO025', 'NEO026', 'NEO027', 'NEO028', 'NEO029', 'NEO030', 'NEO031', 'NEO032', 'NEO033', 'NEO034', 'NEO035', 'NEO036', 'NEO037', 'NEO039', 'NEO040', 'NEO043', 'NEO044', 'NEO045', 'NEO047', 'NEO048', 'NEO050', 'NEO051', 'NEO057', 'NEO058', 'NEO059', 'NEO060', 'NEO061', 'NEO062', 'NEO064', 'NEO065', 'NEO066', 'NEO068', 'NEO069', 'NEO070', 'NEO071', 'NEO072', 'NEO073', 'NEO074', 'NVS001', 'NVS002', 'NVS003', 'NVS004', 'NVS005', 'NVS006', 'NVS007', 'NVS008', 'NVS009', 'NVS010', 'NVS011', 'NVS012', 'NVS013', 'NVS014', 'NVS015', 'NVS016', 'NVS017', 'NVS018', 'NVS019', 'NVS020', 'PNL001', 'PNL002', 'PNL005', 'PNL006', 'PNL007', 'PNL008', 'PNL010', 'PNL011', 'PNL012', 'PNL013', 'PRG001', 'PRG002', 'PRG003', 'PRG004', 'PRG005', 'PRG006', 'PRG008', 'PRG009', 'PRG011', 'PRG012', 'PRG013', 'PRG014', 'PRG015', 'PRG016', 'PRG017', 'PRG018', 'PRG019', 'PRG020', 'PRG022', 'PRG023', 'PRG024', 'PRG027', 'PRG028', 'PRG029', 'PRG030', 'RSP001', 'RSP002', 'RSP003', 'RSP004', 'RSP005', 'RSP006', 'RSP007', 'RSP008', 'RSP009', 'RSP010', 'RSP011', 'RSP012', 'RSP013', 'RSP014', 'RSP016', 'RSP017', 'SKN001', 'SKN002', 'SKN003', 'SKN004', 'SKN005', 'SKN007', 'SYM001', 'SYM002', 'SYM003', 'SYM004', 'SYM005', 'SYM006', 'SYM007', 'SYM008', 'SYM010', 'SYM011', 'SYM012', 'SYM013', 'SYM014', 'SYM015', 'SYM016', 'SYM017', 'Analgesic Opioid Agonists', 'Gastric Acid Secretion Reducing Agents - Proton Pump Inhibitors (PPIs)', 'Antianxiety Agent - Benzodiazepines', 'Anticonvulsant - GABA Analogs', 'Glucocorticoids', 'Antihyperlipidemic - HMG CoA Reductase Inhibitors (statins)', 'Antidepressant - Selective Serotonin Reuptake Inhibitors (SSRIs)', 'NSAID Analgesics (COX Non-Specific) - Propionic Acid Derivatives', 'Laxative - Saline and Osmotic', 'ACE Inhibitors', 'Analgesic or Antipyretic Non-Opioid', 'Skeletal Muscle Relaxant - Central Muscle Relaxants', 'Antiemetic - Selective Serotonin 5-HT3 Antagonists', 'Insulin Analogs - Long Acting', 'Asthma/COPD Therapy - Beta 2-Adrenergic Agents, Inhaled, Short Acting', 'Salicylate Analgesics', 'Antipsychotic -Atypical Dopamine-Serotonin Antag-Dibenzothiazepine Der', 'Dermatological - Topical Local Anesthetic Amides', 'Antidepressant - Serotonin-2 Antagonist-Reuptake Inhibitors (SARIs)', 'Diuretic - Loop', 'Beta Blockers Cardiac Selective', 'Calcium Channel Blockers - Dihydropyridines', 'Medical Supplies and DME - Miscellaneous Other', 'Antianxiety Agent - Antihistamine Type', 'Gastric Acid Secretion Reducers - Histamine H2-Receptor Antagonists', 'Laxative - Surfactant', 'Thyroid Hormones - Synthetic T4 (Thyroxine)', 'Cephalosporin Antibiotics - 1st Generation', 'Medical Supplies and DME - Blood Glucose Tests', 'Alpha-Beta Blockers', 'Nasal Corticosteroids', 'Anticoagulants - Coumarin', 'Insulin Analogs - Rapid Acting', 'Antibacterial Folate Antagonist - Other Combinations', 'Analgesic Opioid Hydrocodone Combinations', 'Antidepressant - Serotonin-Norepinephrine Reuptake Inhibitors (SNRIs)', 'Dermatological - Glucocorticoid', 'Aminopenicillin Antibiotic - Beta-lactamase Inhibitor Combinations', 'Tetracycline Antibiotics', 'Antihistamines - 2nd Generation', 'Laxative - Stimulant', 'Minerals and Electrolytes - Iron', 'Asthma/COPD Therapy - Beta Adrenergic-Glucocorticoid Combinations', 'Direct Factor Xa Inhibitors', 'Asthma Therapy - Leukotriene Receptor Antagonists', 'Minerals and Electrolytes - Potassium, Oral', 'Antiprotozoal-Antibacterial 1st Generation 2-methyl-5-nitroimidazole', 'Smoking Deterrents - Nicotine-Type', 'Medical Supplies and DME - Insulin Needles-Syringes and Admin Supplies', 'Vitamins - B-1, Thiamine and Derivatives', 'Fluoroquinolone Antibiotics', 'Antianginal - Coronary Vasodilators (Nitrates)', 'Gastrointestinal Prokinetic Agents - D2 Antagonist/5-HT4 Agonists', 'Antidepressant - Alpha-2 Receptor Antagonists (NaSSA)', 'Antidepressant-Norepinephrine and Dopamine Reuptake Inhibitors (NDRIs)', 'Asthma/COPD Therapy - Beta Adrenergic-Anticholinergic Combinations', 'Hypnotics - Melatonin - Single Agents', 'Vitamins - B-12, Cyanocobalamin and derivatives', 'Anticonvulsant - Carboxylic Acid Derivatives', 'Angiotensin II Receptor Blockers (ARBs)', 'Medical Supplies and DME - Glucose Monitoring Test Supplies', 'Dermatological - NSAID Single Agents', 'Antidepressant-Tricyclics and Related (Non-Select Reuptake Inhibitors)', 'Analgesic Opioid Oxycodone Combinations', 'Dermatological - Antifungal Amphoteric Polyene Macrolides', 'Antihistamine - 1st Generation - Ethanolamines', 'Cephalosporin Antibiotics - 2nd Generation', 'Platelet Aggregation Inhibitors - Thienopyridine Agents', 'Vitamins - D Derivatives', 'Bipolar Therapy Agents - Atypical Antipsychotics', 'Migraine Therapy - Selective Serotonin Agonists 5-HT(1)', 'Peripheral Alpha-1 Receptor Blockers', 'Antitussives - Non-Opioid', 'Dermatological - Antifungal Imidazole and Related Agents', 'Asthma Therapy - Inhaled Corticosteroids (Glucocorticoids)', 'Diuretic - Aldosterone Receptor Antagonist, Non-selective', '*Unspecified', 'Beta Blockers Non-Cardiac Selective', 'Insulin Response Enhancers - Biguanides', 'Analgesic Opioid Codeine Combinations', 'Low Molecular Weight Heparins', 'Multivitamins', 'Anticonvulsant - Monosaccharide Derivatives', 'Diuretic - Thiazides and Related', 'Herpes Antiviral Agent - Purine Analogs', 'Calcium Channel Blockers - Benzothiazepines', 'Anticonvulsant - Phenyltriazine Derivatives', 'Antacid - Magnesium', 'Lincosamide Antibiotics', 'Opioid Reversal Agents - Opioid Antagonists', 'Expectorants - Single Agents, General', 'Antihyperglycemic - Dipeptidyl Peptidase-4 (DPP-4) Inhibitors', 'Prenatal Vitamins and Minerals', 'Anticonvulsant - Pyrrolidine Derivatives', 'Phosphate Binders', 'Urinary Analgesics', 'Urinary Antispasmodic - Smooth Muscle Relaxants', 'Antihyperglycemic - Sulfonylurea Derivatives', 'Laxative - Saline/Osmotic Mixtures', 'Bipolar Therapy Agents - Lithium', 'Macrolide Antibiotics', 'Antipsychotic - Atypical Dopamine-Serotonin Antag- Benzisoxazole Deriv', 'Immunosuppressive - Calcineurin Inhibitors', 'Agents for Opioid Withdrawal, Opioid-Type', 'Minerals and Electrolytes - Calcium Replacement/Vitamin D Combinations', 'Opioid Antitussive-Expectorant Combinations', 'Multivitamin and Mineral Combinations', 'Antidiarrheal - Antiperistaltic Agents', 'Contraceptive Oral - Monophasic', 'Intestinal Flora Modifiers', 'Antipsychotic - Phenothiazines, Piperazine', 'Antifungal - Triazoles', 'Dietary Product - Dietary Supplements', 'Asthma/COPD - Anticholinergic Agents, Inhaled Long Acting', 'Antipsychotic -Atypical Dopamine-Serotonin Antag-Thienobenzodiazepines', 'Antimalarials', 'Hyperuricemia Therapy - Xanthine Oxidase Inhibitors', 'Glycopeptide Antibiotics', 'Direct Acting Vasodilators', 'Dermatological - Antifungal-Glucocorticoid Combinations', 'IBS Agent - Guanylate Cyclase-C (GC-C) Agonists', 'Dermatological - Antibacterial Other', 'Cardiovascular Sympathomimetic - Anaphylaxis Therapy Single Agents', 'Cephalosporin Antibiotics - 3rd Generation', 'Aminopenicillin Antibiotic', 'GI Antispasmodic - Synthetic Tertiary Amines', 'Gout Acute Therapy - Antimitotics', 'NSAID Analgesics (COX Non-Specific) - Oxicam Derivatives', 'Analgesic or Antipyretic Non-Opioid/Sedative Combinations', 'Antihyperlipidemic - Fibric Acid Derivatives', 'Antiemetic - Antihistamines', 'Antiarrhythmic - Class III', 'Antihyperglycemic, Incretin Mimetic,GLP-1 Receptor Agonist Analog-Type', 'Antacid - Calcium', 'NSAID Analgesic, Cyclooxygenase-2 (COX-2) Selective Inhibitors', 'Antipsychotic - Atypical Dopamine-Serotonin Antag- Benzisothiazolones', 'Antianxiety Agent - Non-Benzodiazepine', 'Mouth and Throat - Antifungals', 'Erectile Dysfunction (ED) Drugs-Sel.cGMP Phosphodiesterase Type5 Inhib', 'Immunosuppressive - Inosine Monophosphate Dehydrogenase Inhibitors', 'Sodium Chloride, Parenteral', 'Antiparkinson Therapy - Anticholinergic Agents', 'CNS Stimulant - Amphetamine Combinations', 'Anticonvulsant - Iminostilbene Derivatives', 'Insulin Analogs - Fixed Combinations', 'Ophthalmic - Anti-inflammatory, Glucocorticoids', 'Influenza Antiviral Agents - Neuraminidase Inhibitors', 'Sedative-Hypnotic - GABA-Receptor Modulators', 'Antiparkinson Therapy - Non-ergot Dopamine Agonist Agents', 'Rifamycins and Related Derivative Antibiotics', 'Peptic Ulcer - Gastric Lumen Adherent Cytoprotectives', 'Non-Opioid Antitussive-Expectorant Combinations', 'Erythropoietins', 'Angiotensin II Receptor Blocker (ARB)-Diuretic Combinations', 'NSAID Analgesics (COX Non-Specific) - Other', 'Inflammatory Bowel Agent - Aminosalicylates and Related Agents', 'Smoking Deterrents - Nicotinic Receptor Partial Agonist, alpha4beta2', 'Antidiarrheal Antiperistaltic-Anticholinergic Combinations', 'Antiretroviral - HIV-1 Integrase Strand Transfer Inhibitors', 'Ophthalmic Antibiotic - Macrolides', 'Protease Inhibitors (Peptidic) Antiretroviral', 'Anorectal - Glucocorticoids', 'Otic (Ear) - Anti-infective-Glucocorticoid Combinations', 'Ophthalmic - Antihistamines', 'Bone Resorption Inhibitors - Bisphosphonates', 'Penicillin Antibiotic - Natural', 'Ophthalmic-Intraocular Pressure Reducing Agents, Prostaglandin Analogs', 'Mouth and Throat - Antiseptics', 'Vaginal Estrogens', 'Vaginal Antiprotozoal-Antibacterial - Nitroimidazole Derivatives', 'Anticonvulsant - Hydantoins', 'Antihyperglycemic-Dipeptidyl Peptidase-4(DPP-4)Inhibitor and Biguanide', 'Minerals and Electrolytes - Calcium Replacement', 'Digestive Enzyme Mixtures', 'Calcimimetic, Parathyroid Calcium Receptor Sensitivity Enhancer', 'Antipsychotic - Phenothiazines, Aliphatic', 'Vaginal Antifungal - Triazoles', 'Urinary Antibacterial - Nitrofuran Derivatives', 'Laxative - Bulk Forming', 'ACE Inhibitor and Diuretic Combinations', 'Attention Deficit-Hyperactivity (ADHD) Therapy, Stimulant-Type', 'Ophthalmic Antibiotic - Fluoroquinolones', 'Irritable Bowel Syndrome (IBS) Agents', 'Heparins', 'Dermatological - Local Anesthetic Combinations', 'Alternative Therapy - Unclassified', 'GI Antispasmodic - Belladonna Alkaloids', 'Dermatological - Emollient Mixtures', 'Diuretic - Potassium Sparing-Thiazide and Related Combinations', 'Antihistamine - 1st Generation - Phenothiazines', 'Minerals and Electrolytes - Magnesium', "Alzheimer's Disease Therapy - Cholinesterase Inhibitors", 'Contraceptive Injectable - Progestin', 'Dermatological - Antibacterial Polymyxins and Derivatives', 'Medical Supplies and DME - Incontinence Supplies', 'Antihyperlipidemic - Selective Cholesterol Absorption Inhibitor', 'Antineoplastic - Aromatase Inhibitors', 'Vitamins - C, Ascorbic Acid and Derivatives', 'Antacid - Simethicone Combinations', 'NSAID Analgesics (COX Non-Specific) - Phenylacetic Acid Derivatives', 'B-Complex Vitamin Combinations', 'Acne Therapy Topical - Anti-infective', 'Antihyperglycemic - Sodium Glucose Cotransporter-2 (SGLT2) Inhibitors', 'Protease Inhibitors (Non-Peptidic) Antiretroviral', 'Central Alpha-2 Receptor Agonists', 'Progestins', 'Calcium Channel Blockers - Phenylakylamines', 'Antipsychotic - Butyrophenone Derivatives', 'Antiretroviral - Nucleoside and Nucleotide Analog RTIs Combinations', 'Alcohol Abstinence Therapy - Glutamate and GABA System Type', 'Antihyperlipidemic - Bile Acid Sequestrants','Cardiovascular Sympathomimetics', 'Contraceptive Oral - Progestin', 'Analgesic Opioid Tramadol Combinations', 'Urinary Antispasmodic - Antichol., M(3) Muscarinic Selective (Bladder)', 'Dermatological - Antipruritics Combinations', 'Antiretroviral-Nucleoside, Nucleotide Analogs and Non-Nucleoside RTI', 'Antiemetic - Antihistamine-Vitamin Combinations', 'Antithyroid Agents, Thionamides - Imidazole Derivatives', 'Antiretroviral - Nucleoside Reverse Transcriptase Inhibitors (NRTI)', 'Human Insulins - Short Acting', 'Scabicide and Pediculicide Single Agents', 'Medical Supplies and DME - Respiratory Therapy Supplies', 'Dermatological - Emollients', 'Ophthalmic - Anti-inflammatory, Immunomodulators', 'Gallstone Solubilizing (Litholysis) Agents', 'Dermatological - Protectants', 'Insulin Response Enhancers - Thiazolidinediones (PPAR-gamma agonists)', 'Inflammatory Bowel Agent - Tumor Necrosis Factor Alpha Blockers', 'B-Complex Vitamins', 'Antiemetic - Cannabinoid Type', 'Antiparkinson - Dopaminerg-Peripheral Dopa-decarboxylase Inhibit Comb', 'Systemic Sympathomimetic Decongestants', 'Medical Supplies and DME - Needles and Syringes', 'Overactive Bladder Agents - Beta -3 Adrenergic Receptor Agonist', 'Dermatological - Burn Products Anti-infective', 'Estrogens', 'Contraceptive Oral - Triphasic', 'Nasal Sympathomimetic Decongestants (Intranasal)', 'Artificial Tears and Lubricant Single Agents', 'Platelet Aggregation Inhib - Cyclopentyl-triazolo-pyrimidines (CPTPs)', 'Nutritional Product - Nutritional Therapy', 'Artificial Tears and Lubricant Combinations', 'Antiretroviral-Nucleoside Analogs and Integrase Inhibitor combinations', 'Ophthalmic - Anti-inflammatory, NSAIDs', 'Angiotensin II Receptor Blocker-Neprilysin Inhibitor Comb. (ARNi)', 'Sedative-Hypnotic - Benzodiazepines', 'Antipsychotic - Atypical Dopamine-Serotonin Antag-Dibenzodiazepine Der', 'Androgen - Single Agents', 'Nasal Antihistamines', 'Sedative-Hypnotic - Barbiturates', 'DMARD - Antimetabolites', 'Human Insulins - Intermediate Acting', 'Asthma/COPD - Anticholinergic Agents, Inhaled Short Acting', 'Ophthalmic Antibacterial Mixtures', 'Dermatological - Antifungal Hydroxypyridinone', 'Misc Anti-Infective', 'Vaginal Antifungal - Imidazoles', 'Hyperparathyroid Treatment Agents - Vitamin D Analog-Type', 'Minerals and Electrolytes - Iron Combinations', 'Otic (Ear) - Wax Removers-Softeners', 'Dermatological Irritants-Counter-Irritant Combinations', 'Sodium Chloride Flushes', "Dermatological - Antineoplastic or Premalignant Lesions - NSAID's", 'Vitamins - B-6, Pyridoxine and Derivatives', 'Opioid Antitussive-Anticholinergic Combinations', 'Medical Supply, FDB Superset', 'B-Complex Vitamins and Combinations', 'Mineralocorticoids', "Alzheimer's Disease Therapy - NMDA Receptor Antagonists", 'Dermatological - Glucocorticoid-Emollient Combinations', 'Dermatological - Enzymes', 'Ophthalmic - Intraocular Pressure Reducing Agents, Beta-blockers', 'Antidepressant - SSRI and 5HT1A Partial Agonist', 'Anticonvulsant - Functionalized Amino Acid', 'Bulk Chemicals', 'Mu-Opioid Receptor Antagonists, Peripherally-Acting', 'Acne Therapy Topical - Retinoids and Derivatives', 'Local Anesthetic - Amides', 'Analgesic Opioid Partial-Mixed Agonists', '2nd Generation Antihistamine-Decongestant Combinations', 'Anticonvulsant - Benzodiazepines', 'Vitamins - Biotin', 'Antibacterial Other', 'Antiemetic - Phenothiazines', 'NSAID Analgesics, (COX Non-Specific) - Indole Acetic Acid Derivatives', 'Thrombin Inhibitor - Selective Direct and Reversible', 'Minerals and Electrolytes - Zinc', 'Agents to treat Hypoglycemia (Hyperglycemics)', 'Antiprotozoal Agents - Other', 'Antineoplastic - Thalidomide Analogs', 'Antiemetic - Anticholinergics', 'Vitamins - Folic Acid and Derivatives', 'Urinary Alkalinizer - Citrates', 'Carbapenem Antibiotics (Thienamycins)', 'GI Antispasmodic Combinations Other', 'Narcolepsy Therapy Agents - Non-Sympathomimetic', 'Contraceptives - Intravaginal, Systemic - Estrogen and Progestin Comb.', 'Ophthalmic Antibiotic - Aminoglycosides', 'Mouth and Throat - Local Anesthetic Others', 'Electrolyte Depleters - Ion Exchange Resin', 'Gastrointestinal Antiflatulents', 'Vitamins - E', 'Human Insulins - Fixed Combinations', 'Acne Therapy Topical - Anti-infective-Keratolytic Combinations', 'Contraceptive Intrauterine - Progesterone IUD', 'Indirect Factor Xa Inhibitors', 'Dermatological Irritants-Counter-Irritant Single Agents', 'Inflammatory Bowel Agent - Glucocorticoids', 'Contraceptive Oral - Biphasic', 'Dermatological - Antifungal Allylamines', 'Antiseptic - Biguanides', 'Antidepressant - SSRI and Serotonin (5-HT) Receptor Modulator', 'Diuretic - Carbonic Anhydrase Inhibitors', 'Antiseptic - Alcohols', 'Ophthalmic - Beta blockers-Carbonic Anhydrase Inhibitor Combinations', 'Antidiarrheal - Bismuth Agents', 'Antiarrhythmic - Class II', 'Alternative Therapy - Antiarthritics', 'Antiretroviral-Nucleoside Reverse Transcriptase Inhibitors (NRTI) Comb', 'ACE Inhibitor and Calcium Channel Blocker Combinations', 'NSAID Analgesics, (COX Non-specific) - Indole Acetic Acid Derivatives', 'Antineoplastic - Protein-Tyrosine Kinase Inhibitors', 'Vaccine Viral - Influenza A and B', 'Medical Supplies and DME - Gauze Pads and Dressings', 'Estrogen-Progestin', 'Otic (Ear) - Fluoroquinolones', 'Antileprotic - Sulfone Agents', 'Ophthalmic - Beta blockers-Adrenergic Combinations', 'Antiarrhythmic - Class Ic', 'Gastric Mucosa - Cytoprotective Prostaglandin Analogs', 'Antihistamine - 1st Generation - Piperidines', 'Antiseptic - Chlorine Releasing', 'Attention Deficit-Hyperact. Disorder (ADHD)- alpha-2 Receptor Agonist', 'Dermatological - Rosacea Therapy, Topical', 'Angiotensin II Receptor Blocker (ARB)-Calcium Channel Blocker Comb.', 'Antihyperlipidemic Agents - Dietary Source Combinations', 'Dermatological - Calcineurin Inhibitors', 'Asthma Therapy - Xanthines', 'Vaccine Bacterial - Gram Positive Cocci', 'Salicylate Analgesic Combinations', 'Minerals and Electrolytes - Sodium Chloride, Oral', 'Minerals and Electrolytes - Calcium Replacement Combinations', 'Non-Opioid Antitussive-Antihistamine Combinations', 'Emergency Contraceptives', 'Dermatological - Keratolytic-Antimitotic Single Agents', 'Dermatological - Astringent Combinations', 'Antineoplastic - Antiandrogens', 'Antineoplastic - Antimetabolite - Pyrimidine Analogs', 'Vaccine Viral - Varicella', 'Digitalis Glycosides', 'Antidiuretic and Vasopressor Hormones', 'Dermatological - Antiseborrheic', 'GI Antispasmodic - Quaternary Ammonium Compounds', 'Attention Deficit-Hyperactivity Disorder (ADHD) Therapy, NRI-Type', 'Anorexiants', 'Dermatological - Antibacterial Mixtures', 'Ophthalmic-Intraocular Press. Reducing, Sel. Alpha Adrenergic Agonists', 'Hypnotics - Melatonin M1/M2 Receptor Agonists', 'Non-Opioid Antitussive-1st Gen.Antihistamine-Decongestant Combinations', 'Nasal Antihistamine and Anti-inflammatory Steroid Combinations', 'Thyroid Hormones - Animal Source (Porcine)', 'Vaginal Antibacterial - Lincosamides', 'Nasal Anticholinergics', 'Dermatological - Protectant Combinations', 'Laxative - Stimulant and Surfactant Combinations', 'Pediatric Vitamins', 'Prostatic Hypertrophy Agent-Type I and II 5-alpha Reductase Inhibitors', 'Ophthalmic - Carbonic Anhydrase Inhibitors', 'Antineoplastic - Selective Estrogen Receptor Modulators (SERMs)', 'Hepatitis B Treatment- Nucleoside Analogs (Antiviral)', 'Antineoplastic - Antimetabolite - Purine Analogs', 'Antihistamine Combinations', 'Oxazolidinone Antibiotics', 'Antianginal and Anti-ischemic Agents, Non-hemodynamic', 'Opioid Antitussive-1st Generation Antihistamine Combinations', 'Vitamins - B Preparation Combinations', 'Dental Product - Fluoride Preparations', 'Otic (Ear) - Local Anesthetic-Analgesic Combinations', 'Antiretroviral - Non-Nucleoside Reverse Transcriptase Inhib (NNRTI)', 'DMARD - Pyrimidine Synthesis Inhibitors', 'Antiarrhythmic - Class IV', 'Minerals and Electrolytes - Phosphate', 'Alcohol Abstinence Therapy - Opioid Receptor Antagonist-Type', 'Penicillin Antibiotic - Penicillinase-resistant', 'Migraine Therapy - Ergot Alkaloids and Derivatives', 'Nutritional Product - Protein Replacements', 'Dermatological - Antibacterial Aminoglycosides', 'Sedative-Hypnotic - Orexin Receptor Antagonist', 'Thyroid Hormones - Synthetic T3 (Triiodothyronine)', 'Antacid - Antacid Combinations', 'Nasal Antibiotics', 'Ophthalmic - Antibacterial-Glucocorticoid Combinations', 'Acne Therapy Topical - Keratolytic', 'Dermatological - Antipsoriatic Agents Topical', 'Asthma/COPD -  Phosphodiesterase-4 (PDE4) inhibitors', 'Contraceptive Implant - Progestin', 'Antitubercular - Rifamycin and Derivatives', 'Nasal Moisturizers', 'Cardiac Selective Beta Blocker-Thiazide Diuretic and Related Comb.', 'Platelet Aggregation Inhibitors - Phosphodiesterase III Inhibitors', 'Wound Care - Dressings', 'Vitamins - B-2, Riboflavin and Derivatives', 'Penicillin Antibiotic, Extended-spectrum and Beta-lactamase Inhib Comb', 'Antihyperlipidemic - Nicotinic Acid Derivatives', 'Ophthalmic - Anticholinergics', 'RANK ligand (RANKL) inhibitor, MC Antibody', 'Otic (Ear) - Glucocorticoids', 'Alternative Therapy - Pineal Hormone Agents', 'Antiretroviral Combinations - Protease Inhibitors', 'Multiple Sclerosis Agent - Others', 'Chelating Agents - Iron', 'Mouth and Throat - Lozenges', 'Prostatic Hypertrophy Agent - alpha-1-Adrenoceptor Antagonists', 'Anticonvulsant - Barbiturates and Derivatives', 'Antihyperglycemic - Sulfonylurea and Biguanide Combinations', 'Immunosuppressive - Mammalian Target of Rapamycin (mTOR) Inhibitors', 'Mouth and Throat - Saliva Stimulants', 'Gastric Acid Secretion Reducing-H2 Antagonist and Antacid Combinations', 'Dermatological - Antiviral, Herpes', 'Diuretic - Aldosterone Receptor Antagonist, Selective', 'Salicylate Analgesic and Sedative Combinations', 'Alternative Therapy - Antioxidant', 'Hepatitis C - NS5B Polymerase and NS5A Inhibitor Combinations', 'Antitubercular - Isonicotinic Acid Derivatives', 'Antineoplastic - Antimetabolite - Folic Acid Analogs', 'Sedative-Hypnotic - Antihistamines', 'Anorectal - Hemorrhoidal Combinations Other', 'Pharmaceutical Adjuvant - Inhalation Vehicles', 'Neuromuscular Blocker - Neurotoxins', 'Antiprotozoal-Antibacterial 2nd Generation 2-methyl-5-nitroimidazole', 'Hematorheologic Agents', 'Appetite Stimulants - Progestin Hormone Type', 'Antifungal - Amphoteric Polyene Macrolides', 'Antihyperglycemic - Meglitinide Analogs', 'Angiotensin II Receptor Blocker (ARB)-Calcium Channel Blocker-Diuretic', 'Digestive Enzymes', 'Medical Supplies and DME - Nebulizers', 'Dermatological Antipruritics - Antihistamines', 'Antiarrhythmic - Class Ib', 'Anorectal - Hemorrhoidal Rectal Glucocorticoid-Local Anesthetic Comb', 'Medical Supplies and DME - Gauze Bandages', 'Cyclic Lipopeptide Antibiotics', 'Peptic Ulcer-Treatment H. Pylori-Proton Pump Inhibitor and Antibiotics', 'Pharmaceutical Adjuvant - Cream/Ointment Vehicles', 'Amino Acids, Single Ingredient, Oral (non-injectable)', 'Ophthalmic - Adrenergic-Carbonic Anhydrase Inhibitor Combinations', 'Laxative - Lubricant', 'Mouth and Throat - Local Anesthetic Amides', 'Misc Anti-Infective Combinations', 'DMARD - Anti-inflammatory Tumor Necrosis Factor Inhibiting Agents', 'Platelet Aggregation Inhibitor Combinations', 'Hepatitis C - Nucleoside Analogs', 'Antihyperlipidemic HMG CoA Reduct Inhib and Calcium Channel Blocker', 'Antacid - Bicarbonate', 'Pediatric Vitamins and Mineral Combinations', 'Colonic Acidifier (Ammonia Inhibitor)', 'Mouth and Throat - Artificial Saliva', 'Vitamins - A', 'Antihyperglycemic - Alpha-Glucosidase Inhibitors', 'Asthma/COPD Therapy - Beta Adrenergic Agents', 'Methotrexate Rescue Agents - Folic Acid Antagonist Type', 'Medical Supplies and DME - Elastic Bandages and Supports', 'Mucolytics', 'Vitamins - K, Phytonadione and Derivatives', 'DMARD - Phosphodiesterase-4 (PDE4) Inhibitors', 'Antineoplastic - Progestins', 'Dermatological - Immunomodulator - Imidazoquinolinamines', 'Mouth and Throat - Local Anesthetic Esters', 'Antiretroviral - Nucleotide Analog Reverse Transcriptase Inhibitors', 'Medical Supplies and DME - Gloves', 'Irrigation Solutions', 'Prolactin Inhibitor - Ergot Derivative Dopamine Receptor Agonists', 'Asthma Therapy - Monoclonal Antibodies to Immunoglobulin E (IgE)', 'Aminoglycoside Antibiotic', 'Dextrose Solutions', 'Antiretroviral - Integrase Inhibitors', 'Antihyperlipidemic-HMG CoA Reduct Inhib and Cholesterol Absorp Inhibit', 'Acne Therapy Systemic - Retinoids and Derivatives', 'Somatostatic Agents', 'Asthma/COPD Therapy - Beta 2-Adrenergic Agents, Inhaled, Long Acting', 'Antiretroviral - CCR5 Co-Receptor Antagonist', 'Immune Globulin - gamma globulin (IgG), human', 'Anthelmintic Agents - Macrocyclic Lactones', 'Salicylate Analgesics, Buffered', 'Miotics - Direct Acting', 'Ophthalmic - Hyperosmolar Agents', 'Ophthalmic Antibiotic - Dehydropeptidase Inhibitors', 'Antiparkinson Therapy - Monoamine Oxidase Inhibitor(MAO-B)', 'Ophthalmic - Antihistamine-Decongestant Combinations', 'Antiemetic - Substance P-Neurokinin 1 (NK1) Receptor Antagonists', 'Musculoskeletal Therapy Agent - Viscosupplements', 'Decongestant-Expectorant Combinations', 'Vaccine Viral - Human Papillomavirus (HPV) Vaccines', 'Dermatological - Antifungal Triazole', 'Antineoplastic - LHRH (GnRH) Agonist Analog Pituitary Suppressants', 'Mouth and Throat - Glucocorticoids', 'Antineoplastic - Alkylating Agent - Nitrogen Mustards', 'Dermatological - Antipsoriatic Agents Systemic, Vitamin A Derivatives', 'Cephalosporin Antibiotics - 4th Generation', 'Dermatological - Astringents', 'Urinary Retention Therapy - Parasympathomimetic Agents', 'Hemostatic Systemic - Antifibrinolytic Agents', 'Multiple Sclerosis Agent - Interferons', 'Anorectal - Hemorrhoidal Single Agents Other','Minerals and Electrolytes - Oral Electrolytes', 'Antihyperlipidemic Agents - Dietary Source', 'Granulocyte Colony-Stimulating Factor (G-CSF)', 'Asthma/COPD Therapy - Anticholinergic Agents', 'Otic (Ear) - Anti-infectives other', 'Antineoplastic - CD20 Specific Recombinant Monoclonal Antibody Agents', 'Gastric Acid Secretion Reducing-Proton Pump Inhibitor and Antacid Comb', 'Dermatological - Antifungal Thiocarbamate', 'Dermatological - Antiperspirants', 'Antiparkinson - Dopaminergic-Periph COMT-Dopa-decarboxylase Inhib Comb', 'Urinary Acidifier - Phosphates', 'Antineoplastic - mTOR Kinase Inhibitors', 'Non-Opioid Antitus-1st Gen Antihist.-Decongest-Analgesic,Non-Salicylat', 'Oxytocic - Ergot Alkaloids', 'Hepatitis C - NS3/4A Serine Protease Inhibitors', 'Laxative - Stimulant and Saline/Osmotic Combinations', 'Medical Supplies and DME - Ostomy Supplies', 'Antineoplastic - Antimetabolite - Urea Derivatives', 'Opioid Antitussive-Decongestant-Expectorant Combinations', 'Mouth and Throat - Mouthwashes', '1st Generation Antihistamine-Decongestant Combinations', 'Movement Disorder Therapy - Restless Legs Syndrome', 'Medical Supplies and DME - Adhesive Tape', 'CNS Stimulant - Amphetamines', 'Hair Growth Agents - Type II 5-Alpha Reductase Inhibitors', 'Antifungal other', 'Herpes Antiviral Agent - Thymidine Analogs', 'Antibacterial Folate Antagonist Others', 'Antineoplastic - Multikinase Inhibitors', 'Minerals and Electrolytes - Trace Minerals', 'Antiprotozoal Agents (antiparasitic) - 5-Nitrothiazolyl Derivatives', 'Periodontal Product - Tetracycline-Type, Collagenase Inhibitors', 'Erectile Dysfunction (ED) Drugs - Prostaglandins', 'NSAID Analgesic and Hypnotic (1st Generation Antihistamine) Combins', 'BPH Agent- 5-alpha Reductase Inhib and alpha-1 Adrenoceptor Antag Comb', 'Acne Therapy Topical - Retinoid Combinations Other', 'Antihistamine - 1st Generation - Alkylamines', 'Antifungal - Imidazoles', 'Decongestant-Analgesic, NSAID Cox Non-specific', 'Antimalarial Combinations', 'Platelet Aggregation Inhib-PDEsterase and Adenosine deaminase Inhibitr', 'Dermatological - Glucocorticoid-Local Anesthetic Combinations', 'Contraceptive Intrauterine - Copper IUD', 'LHRH (GnRH) Agonist Analog Pituitary Suppressants', 'Chemicals - Fixed Oils', 'Diuretic - Potassium Sparing', 'Medical Supplies and DME - Humidifiers', 'Antiparkinson Adjuvant - Peripheral COMT Inhibitors', 'Antithyroid Agents, Thionamides - Thiouracil Derivatives', 'Toxoid Vaccine Combinations', 'NSAID Analgesic and Prostaglandin Analog Combinations', 'Analgesic,Non-Salicylate and 1st Generation Antihistamine Combinations', 'Fertility Enhancer - Luteal Phase Supporting, Progesterone-type', 'Antihyperglycemic - Thiazolidinedione and Biguanide Combinations', '1st Generation Antihistamine-Decongestant-Analgesic, Non-Salicylate', 'Minerals and Electrolytes - Potassium Combinations', 'CMV Antiviral Agent - Nucleoside Analogs', 'Renin Inhibitor, Direct', 'Human Albumin', 'Dermatological - Ornithine Decarboxylase (ODC) Inhibitors', 'Hyperuricemia Therapy - Uricosurics', 'Vitamins - B-3, Niacin and Derivatives', 'Vaccine Bacterial - Gram Negative Bacilli (Non-Enteric)', 'Dermatological - Local Anesthetic Gas Single Agents', 'Menopausal Symptoms Suppressant-Selective Estrogen Receptor Modulators', 'Alternative Therapy - Androgenic Agents', 'Dextrose and Sodium Chloride Solutions', 'Urinary Antispasmodic - Anticholinergics, Non-Selective', 'CNS Stimulant - Analeptics, methylxanthine-type', 'Minerals and Electrolytes - Potassium for Injection', 'Non-Opioid Antitussive-Decongestant-Analgesic, Non-Salicylate Comb', 'Dermatological - Antineoplastic Antimetabolites', 'Growth Hormones', 'Contact Lens Preparation - Soft Lens Products', 'Antineoplastic - Epipodophyllotoxins', 'Decongestant-Analgesic, Non-Salicylate Combinations', 'Medical Supplies and DME - IUDs', 'Sulfonamide Antibiotic', 'Antihyperglycemic-Dipeptidyl Peptidase-4 Inhibit and Thiazolidinedione', 'Selective Estrogen Receptor Modulators (SERMs)', 'Cystic Fibrosis - Inhaled Monobactams', 'Antihyperlipidemic - HMG CoA Reductase Inhibitor and Niacin Comb', 'Antineoplastic - Platinum Complexes', 'ed_10331-7_neg', 'ed_882-1_o positive', 'ed_882-1_b positive', 'ed_14578-9_o neg', 'ed_882-1_a positive', 'ed_14578-9_o pos', 'ed_882-1_a negative', 'ed_50957-0_performed', 'ed_14578-9_a pos', 'ed_890-4_pos', 'ed_890-4_neg', 'ed_10331-7_pos', 'ed_882-1_b negative', 'ed_882-1_o negative', 'ed_882-1_ab positive', 'ed_882-1_nan', 'ed_1744-2', 'ed_11580-8', 'ed_57751-0', 'ed_13315-7', 'ed_4542-7', 'ed_20408-1', 'ed_22312-3', 'ed_16128-1', 'ed_50557-8', 'ed_6566-4', 'ed_19123-9', 'ed_2571-8', 'ed_2110-5', 'ed_50551-1', 'ed_4073-3', 'ed_13969-1', 'ed_13536-8', 'ed_5048-4', 'ed_32019-2', 'ed_43304-5', 'ed_777-3', 'ed_785-6', 'ed_48643-1', 'ed_22315-6', 'ed_5195-3', 'ed_3094-0', 'ed_4024-6', 'ed_2500-7', 'ed_50552-9', 'ed_788-0', 'ed_50627-9', 'ed_50558-6', 'ed_53328-1', 'ed_14979-9', 'ed_2157-6', 'ed_43305-2', 'ed_14804-9', 'ed_2951-2', 'ed_5213-4', 'ed_786-4', 'ed_3948-7', 'ed_789-8', 'ed_718-7', 'ed_2093-3', 'ed_30522-7', 'ed_2324-2', 'ed_1975-2', 'ed_664-3', 'ed_50553-7', 'ed_600-7', 'ed_6301-6', 'ed_22314-9', 'ed_2345-7', 'ed_787-2', 'ed_2112-1', 'ed_18262-6', 'ed_2823-3', 'ed_2085-9', 'ed_31204-1', 'ed_2075-0', 'ed_46420-6', 'ed_58413-6', 'ed_33914-3', 'ed_1920-8', 'ed_53525-2', 'ed_4548-4', 'ed_18481-2', 'ed_9317-9', 'ed_626-2', 'ed_2777-1', 'ed_5643-2', 'ed_1962-0', 'ed_50561-0', 'ed_2284-8', 'ed_13457-7', 'past_10331-7_neg', 'past_882-1_o positive', 'past_882-1_b positive', 'past_14564-9_not applicable', 'past_882-1_a positive', 'past_890-4_nan', 'past_14578-9_o pos', 'past_890-4_anti-k', 'past_890-4_cancelled', 'past_10331-7_positive', 'past_882-1_a negative', 'past_50957-0_performed', 'past_14578-9_a pos', 'past_890-4_unidentified igg antibody', 'past_890-4_pos', 'past_890-4_neg', 'past_10331-7_cancelled', 'past_14578-9_a positive', 'past_10331-7_pos', 'past_882-1_b negative', 'past_882-1_o negative', 'past_882-1_ab positive', 'past_10331-7_negative', 'past_882-1_nan', 'past_14565-6_not applicable', 'past_48642-3', 'past_19642-8', 'past_1744-2', 'past_14310-7', 'past_11580-8', 'past_57751-0', 'past_14314-9', 'past_14316-4', 'past_13315-7', 'past_4542-7', 'past_20408-1', 'past_22312-3', 'past_18282-4', 'past_19261-7', 'past_50557-8', 'past_6566-4', 'past_19123-9', 'past_2571-8', 'past_2110-5', 'past_2498-4', 'past_5794-3', 'past_50551-1', 'past_4073-3', 'past_13969-1', 'past_15205-8', 'past_13536-8', 'past_5048-4', 'past_14563-1', 'past_5880-0', 'past_32019-2', 'past_43304-5', 'past_777-3', 'past_785-6', 'past_10352-3', 'past_34713-8', 'past_48643-1', 'past_22315-6', 'past_3094-0', 'past_4024-6', 'past_2500-7', 'past_19295-5', 'past_50552-9', 'past_788-0', 'past_50627-9', 'past_50558-6', 'past_53328-1', 'past_14979-9', 'past_2157-6', 'past_43305-2', 'past_14804-9', 'past_2951-2', 'past_42242-8', 'past_786-4', 'past_3948-7', 'past_789-8', 'past_718-7', 'past_2093-3', 'past_30522-7', 'past_2324-2', 'past_1975-2', 'past_664-3', 'past_14312-3', 'past_50553-7', 'past_6948-4', 'past_1690-7', 'past_600-7', 'past_6301-6', 'past_610-6', 'past_2345-7', 'past_787-2', 'past_2112-1', 'past_18262-6', 'past_19270-8', 'past_2823-3', 'past_2085-9', 'past_2075-0', 'past_46420-6', 'past_58413-6', 'past_33914-3', 'past_1920-8', 'past_53525-2', 'past_20454-5', 'past_4548-4', 'past_18481-2', 'past_626-2', 'past_2777-1', 'past_5643-2', 'past_1962-0', 'past_6463-4', 'past_50561-0', 'past_2284-8', 'past_13457-7', 'past_57901-1', 'past_2890-2', 'past_11277-1'])

In [148]:
# get the indices for the pandas column names 
cat_col_idxs = column_index(data[train_col_mask], train_col_mask.intersection(cat_cols))
normal_col_idxs = column_index(data[train_col_mask], train_col_mask.intersection(normal_cols))
all_other_cols_idxs = column_index(data[train_col_mask], train_col_mask.intersection(all_other_cols))

print("Categorical columns with no Scaling: ", train_col_mask.intersection(cat_cols).tolist())
print("Normal columns with Standard Scaling: ", train_col_mask.intersection(normal_cols).tolist())
print("Non Normal columns with MinMax Scaling: ", train_col_mask.intersection(all_other_cols).tolist())


Categorical columns with no Scaling:  []
Normal columns with Standard Scaling:  []
Non Normal columns with MinMax Scaling:  []


In [135]:
all_other_cols_idxs.shape[0] + normal_col_idxs.shape[0] + cat_col_idxs.shape[0]

0

Acute cystitis                         13695
Urolithiasis                           11556
Abdominal Pain, general                10055
Colitis                                 9204
Constipation                            8679
                                       ...  
Labor/Preterm Labor                        6
Porphyria                                  5
Uremia                                     4
Paroxysmal nocturnal hemoglobinuria        3
Placental Abruption                        2
Name: label, Length: 64, dtype: int64

In [55]:

# fit the indices by data type
# we check if there are any such columns
# if not, this fails silently
if len(normal_col_idxs) > 0:
    std_scaler.fit(X_train.iloc[:, normal_col_idxs])
    X_train_std_scaled = std_scaler.transform(X_train.iloc[:, normal_col_idxs])
    X_test_std_scaled = std_scaler.transform(X_test.iloc[:, normal_col_idxs])
else:
    X_train_std_scaled = np.array([])
    X_test_std_scaled = np.array([])
    
if len(all_other_cols_idxs) > 0:
    minmax_scaler.fit(X_train.iloc[:, all_other_cols_idxs])
    X_train_minmax_scaled = minmax_scaler.transform(X_train.iloc[:, all_other_cols_idxs])
    X_test_minmax_scaled = minmax_scaler.transform(X_test.iloc[:, all_other_cols_idxs])
else:
    X_train_minmax_scaled = np.array([])
    X_test_minmax_scaled = np.array([])
    

In [57]:

# scale/normalize based on the column idxs above
X_train_input = torch.zeros(X_train.values.shape)
X_test_input = torch.zeros(X_test.values.shape)
print(X_train_input.shape)

torch.Size([99168, 2254])


In [85]:

X_train_input[:, cat_col_idxs] = torch.tensor(np.nan_to_num(X_train.iloc[:, cat_col_idxs]),
                                                 dtype=torch.float)

X_train_input[:, normal_col_idxs] = torch.tensor(np.nan_to_num(X_train_std_scaled),
                                                 dtype=torch.float)
X_train_input[:, all_other_cols_idxs] = torch.tensor(np.nan_to_num(X_train_minmax_scaled),
                                                     dtype=torch.float)


X_test_input[:, cat_col_idxs] = torch.tensor(np.nan_to_num(X_test.iloc[:, cat_col_idxs]),
                                                dtype=torch.float)
X_test_input[:, normal_col_idxs] = torch.tensor(np.nan_to_num(X_test_std_scaled),
                                                dtype=torch.float)
X_test_input[:, all_other_cols_idxs] = torch.tensor(np.nan_to_num(X_test_minmax_scaled),
                                                    dtype=torch.float)



In [2]:
X_train_input = X_train_input[:, list(cat_col_idxs) + list(normal_col_idxs) + list(all_other_cols_idxs)]
X_test_input = X_test_input[:, list(cat_col_idxs) + list(normal_col_idxs) + list(all_other_cols_idxs)]

NameError: name 'X_train_input' is not defined

torch.Size([99168])

In [58]:
X_test_input.shape

torch.Size([24792, 2254])

In [37]:
# y_pred = clf.predict_proba(X_test)
out = model(torch.tensor(X_test_input).float().to(device))

/gpfs/milgram/project/rtaylor/vs428/conda_envs/factcheck_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


RuntimeError: mat1 dim 1 must match mat2 dim 0

In [ ]:
test, y_pred =  torch.max(out.data, 1)
y_pred = y_pred.cpu()

In [ ]:
y_test_transformed = le.transform(y_test)

In [ ]:
print(y_pred.shape)
pd.Series(le.inverse_transform(y_pred)).value_counts()

In [ ]:
len(le.classes_)

In [ ]:
# torch.tensor(X_test.fillna(0).to_numpy()).float().to(device)


In [ ]:
# fig, axs = plt.subplots(121, figsize=(14,14))
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(28,14))#sharey=True)

ConfusionMatrixDisplay.from_predictions(
   y_test_transformed, y_pred, ax=ax1, display_labels=le.classes_[:-1], include_values=False, normalize="true")
# for tick in ax.get_xticklabels():
#     tick.set_rotation(45)
ax1.set_yticklabels(ax1.get_yticklabels(),
                   size=10,
)

ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90, 
                   size=10,
                   ha='center')

ConfusionMatrixDisplay.from_predictions(
   y_test_transformed, y_pred, ax=ax2, display_labels=le.classes_[:-1], include_values=False, normalize=None)
# for tick in ax.get_xticklabels():
#     tick.set_rotation(45)
ax2.set_yticklabels(ax2.get_yticklabels(),
                   size=10,
)

ax2.set_xticklabels(ax2.get_xticklabels(), rotation=90, 
                   size=10,
                   ha='center')

plt.show()

# img = plt.imshow(confusion_matrix(y_test, y_pred), interpolation=None, cmap="viridis")
# plt.colorbar(img)
# plt.show()

In [63]:
from torch.nn.parameter import Parameter
from torch.optim import SGD 

dataset = [[torch.rand(5,2), torch.tensor([0,1])]]
model = [Parameter(torch.randn(2, 2, requires_grad=True))]
optimizer = SGD(model, 0.1)

for epoch in range(20):
    for inputx, target in dataset:
        print(inputx)
        optimizer.zero_grad()
        output = model(inputx)
        loss = loss_fn(output, target)
        print(loss)
        loss.backward()
        optimizer.step()
#     scheduler.step()


tensor([[0.0800, 0.6301],
        [0.8544, 0.3274],
        [0.4094, 0.8099],
        [0.5989, 0.8900],
        [0.8066, 0.6506]])


TypeError: 'list' object is not callable

In [11]:
def _effective_num_weighting(beta, samples_per_cls, no_of_classes):
    '''
    Compute the effective number sample weighting as per https://doi.org/10.1109/CVPR.2019.00949

    beta [float] : usually between 0.9 and 0.99, and is the hyperparam for the amount of weighting like inverse freq
    samples_per_cls [list-like] : a list-like object with the number of samples per class that has length no_of_classes
    no_of_classes [int] : the number of classes
    '''
    effective_num = 1.0 - np.power(beta, samples_per_cls)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights) * no_of_classes
    return weights


In [42]:
samples_per_cls = data['label'].value_counts()
no_of_classes = samples_per_cls.size
class_weights = pd.Series(_effective_num_weighting(0.9, samples_per_cls, no_of_classes),
                    index=samples_per_cls.index)

class_weights = pd.Series(compute_class_weight(class_weight="balanced",classes=samples_per_cls.index, y=data['label']),
                    index=samples_per_cls.index )

class_weights = (1 / np.sqrt(samples_per_cls)) * (samples_per_cls.sum() / 10000.0)

In [43]:
class_weights

Abdominal Pain, general       0.069439
Acute cystitis                0.155096
Urolithiasis                  0.168841
Colitis                       0.189188
Constipation                  0.194826
                               ...    
Placental Abruption          12.834129
Chorioamnionitis             18.150200
Placenta accreta             18.150200
HELLP syndrome               18.150200
Malposition of the uterus    18.150200
Name: label, Length: 68, dtype: float64

In [44]:
1/ np.sqrt(samples_per_cls)

Abdominal Pain, general      0.003826
Acute cystitis               0.008545
Urolithiasis                 0.009302
Colitis                      0.010423
Constipation                 0.010734
                               ...   
Placental Abruption          0.707107
Chorioamnionitis             1.000000
Placenta accreta             1.000000
HELLP syndrome               1.000000
Malposition of the uterus    1.000000
Name: label, Length: 68, dtype: float64

# Test Runner

In [26]:

def flatten(t):
    return [item for sublist in t for item in sublist]


In [65]:
import os
import sys
import string 
import random 


# from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight    

import optuna


import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import ConfusionMatrixDisplay

import csv
import ast
from datetime import date, datetime, time, timedelta
from datetime import datetime
import pickle as pkl
import time

from sklearn import preprocessing
from sklearn.metrics import top_k_accuracy_score

import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from joblib import dump, load

import matplotlib.pyplot as plt

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.optim.lr_scheduler import * 

import wandb
import torchmetrics

import logging


In [66]:

################
## Functions ###
################
def _effective_num_weighting(beta, samples_per_cls, no_of_classes):
    '''
    Compute the effective number sample weighting as per https://doi.org/10.1109/CVPR.2019.00949

    beta [float] : usually between 0.9 and 0.99, and is the hyperparam for the amount of weighting like inverse freq
    samples_per_cls [list-like] : a list-like object with the number of samples per class that has length no_of_classes
    no_of_classes [int] : the number of classes
    '''
    effective_num = 1.0 - np.power(beta, samples_per_cls)
    weights = (1.0 - beta) / np.array(effective_num)
    weights = weights / np.sum(weights) * no_of_classes
    return weights

def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]


In [73]:

################
### Settings ###
################

data_fp = f"/gpfs/milgram/project/rtaylor/shared/ABDPain_EarlyDiags/unq_pt_enc_single_label_full_clean_label.csv"
logger_fp = f"/gpfs/milgram/project/rtaylor/shared/ABDPain_EarlyDiags/model_outputs/simple_nn_logging.txt"

# Create W&B study
# wandb.init(project="test-project", entity="decile")
# WANDB_RUN_NAME = wandb.run.name

# logging.basicConfig(filename=logger_fp, 
#                     level=logging.INFO,
#                    format=f'{WANDB_RUN_NAME} | %(asctime)s | %(levelname)s | %(message)s')
# logging.info(f'Data File loc: {data_fp}')


device = 'cuda' if torch.cuda.is_available() else 'cpu'


wandb.config = {
    "learning_rate": 0.00001,
    "lr_weight_decay":0.0,
    "lr_scheduler":False,
    "epochs": 500,
    "batch_size": 256,
    # drop columns that don't have at least this many non-NA values
    "drop_sparse_cols":10,
#     "multilabel":True,
    "dropout":0.2,
    # can be "inverse", None, "effective_sample", or "balanced"
    "class_weight_type":"inverse",
    # The amount to normalize by in inverse class weighting
    "class_weight_inv_lambda":10000.0,
    # only used if class_weight_type is "effective sample"
    "weight_beta" : 0.999,
}

print(wandb.config)

#################
### Data Load ###
#################

le = preprocessing.LabelEncoder()
std_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

data = pd.read_csv(data_fp , nrows=1000)
# data = pd.read_csv(data_fp)
# data = data.sample(50000)

print(f"dataset size: {data.shape}")

data = data.rename(columns={"WikEM_overalltopic" : "label"})
logging.info(f"Data loaded successfully!")

single_support_classes = set(data['label'].value_counts()[data['label'].value_counts() == 1].index)
droppable_rows = data['label'].isin(single_support_classes).sum()
data = data[~data['label'].isin(single_support_classes)]
train_test_stratify = data['label']

# drop all columns that don't have any positive actual values/only have all NaNs
data = data.drop( data.columns[((data.shape[0]- data.isnull().sum()) == 0)], 
                 axis=1, errors='ignore' )

# remove columns that don't have at least N (hyperparam) number of non-NaN values
data = data[ data.columns.intersection(data.columns[(((data.shape[0]- data.isnull().sum())) > wandb.config['drop_sparse_cols'])]) ]
print(f"After dropping sparse columns: {data.shape}")

data.columns = data.columns.str.replace("[|]|<", "leq_")
# logging.info(f"Dropped {droppable_rows} rows for stratified K-fold with {single_support_classes} classes")

N_CLASSES = data['label'].nunique() # 65

# drop EDDisposition, ID, and label columns
non_train_col_mask = data.columns[data.columns.str.contains("EdDisposition_")].union(data.columns[:3], sort=False).union(pd.Index(['label']), sort=False)
train_col_mask = data.columns.difference(non_train_col_mask, sort=False)

le.fit(data['label'])
# get the 1/class_size weights for training
if wandb.config['class_weight_type']:
    samples_per_cls = data['label'].value_counts()

#    class_weights = torch.tensor((1/data['label'].value_counts()).reindex(le.classes_).values).float()
#    class_weights = class_weights.to(device)
#     from sklearn.utils import compute_class_weight
#     class_weights = compute_class_weight(
#                                         class_weight="balanced",
#                                         classes=le.classes_,
#                                         y=data['label']                                                
#                                     )
#     class_weights = torch.tensor(class_weights).float().to(device)
    if wandb.config['class_weight_type'] == "effective_sample":
        beta = wandb.config['weight_beta']
        no_of_classes = samples_per_cls.size
        class_weights = pd.Series(_effective_num_weighting(beta, samples_per_cls, no_of_classes),
                            index=samples_per_cls.index)
    
    elif wandb.config['class_weight_type'] == "balanced":
        class_weights = pd.Series(compute_class_weight(class_weight="balanced",classes=samples_per_cls.index, y=data['label']),
                            index=samples_per_cls.index )
    elif wandb.config['class_weight_type'] == "inverse":
        # multiply by total/2 as per tensorflow core example imbalanced classes
        class_weights = (1 / samples_per_cls) * (samples_per_cls.sum() / wandb.config['class_weight_inv_lambda'])

    print(class_weights)
    class_weights = torch.tensor(class_weights.values).float().to(device)

#######
# Scale variables either MinMax/Standard depending on if they are Normal-ish or not
# Note: We do the fitting after train_test_split since we only fit on training data
#######

# first separate out the corresponding cols
# all those that take avgs probably are normal
avg_cols = data.columns[data.columns.str.contains("_avg")].union(pd.Index(["age"]))
# same with vitals
vital_cols = pd.Index(['last_SpO2', 'last_Temp', 'last_Patient Acuity', 'last_Pulse', 'last_Pain Score', 'last_Resp', 'last_BP_Systolic', 'last_BP_Diastolic', 'ed_SpO2', 'ed_Temp', 
 'ed_Patient Acuity', 'ed_Pulse', 'ed_Pain Score', 'ed_Resp', 'ed_BP_Systolic', 'ed_BP_Diastolic'])
normal_cols = avg_cols.union(vital_cols)

# all the other columns are not normal, since they are all counts, which have a long tail (likely Poisson)
# except for the purely categorical ones
cat_col_headers = ["EdDisposition_","DepartmentName_","Sex_",
                     "GenderIdentity_","FirstRace_","Ethnicity_",
                     "PreferredLanguage_","SmokingStatus_",
                     "AcuityLevel_","FinancialClass_","CC_"]

cat_cols = pd.Index(flatten([data.columns[data.columns.str.contains(col)].tolist() for col in cat_col_headers]))

all_other_cols = data.columns.difference(cat_cols, sort=False)
all_other_cols = all_other_cols.difference(normal_cols, sort=False)
# remove label as well, since we don't scale it
all_other_cols = all_other_cols.difference(pd.Index(['label']), sort=False)


# Create a dummy indices variable to get the indices
indices = range(data.shape[0])
X_train, X_test, y_train, y_test, idxs_train, idxs_test = train_test_split(data[train_col_mask],
                                                    data['label'],
                                                    indices,
                                                    stratify=train_test_stratify, # we would like to stratify by label, but we have a few that only have one instance
                                                   test_size=0.2, random_state=314, 
                                                   )
print("le shape", le.classes_.shape)
logging.info(f"Created training/test sets")

print(data[train_col_mask].shape)
print(X_train.shape)

# get the indices for the pandas column names 
cat_col_idxs = column_index(data[train_col_mask], train_col_mask.intersection(cat_cols))
normal_col_idxs = column_index(data[train_col_mask], train_col_mask.intersection(normal_cols))
all_other_cols_idxs = column_index(data[train_col_mask], train_col_mask.intersection(all_other_cols))

print("Categorical columns with no Scaling: ", train_col_mask.intersection(cat_cols).tolist())
print("Normal columns with Standard Scaling: ", train_col_mask.intersection(normal_cols).tolist())
print("Non Normal columns with MinMax Scaling: ", train_col_mask.intersection(all_other_cols).tolist())

# fit the indices by data type
std_scaler.fit(X_train.iloc[:, normal_col_idxs])
minmax_scaler.fit(X_train.iloc[:, all_other_cols_idxs])

# scale/normalize based on the column idxs above
X_train_input = torch.zeros(X_train.values.shape)
X_test_input = torch.zeros(X_test.values.shape)

### TODO: The error is in this line where the transform function expects what we trained on, which we aren't using here. terrible. 
X_train_input[:, cat_col_idxs] = torch.tensor(np.nan_to_num(X_train.iloc[:, cat_col_idxs]),
                                                 dtype=torch.float)
X_train_input[:, normal_col_idxs] = torch.tensor(np.nan_to_num(X_train.iloc[:, normal_col_idxs]),
                                                 dtype=torch.float)
X_train_input[:, all_other_cols_idxs] = torch.tensor(np.nan_to_num(X_train.iloc[:, all_other_cols_idxs]),
                                                     dtype=torch.float)

X_test_input[:, cat_col_idxs] = torch.tensor(np.nan_to_num(X_test.iloc[:, cat_col_idxs]),
                                                dtype=torch.float)
X_test_input[:, normal_col_idxs] = torch.tensor(np.nan_to_num(X_test.iloc[:, normal_col_idxs]),
                                                dtype=torch.float)
X_test_input[:, all_other_cols_idxs] = torch.tensor(np.nan_to_num(X_test.iloc[:, all_other_cols_idxs]),
                                                    dtype=torch.float)

# we need to store this value for the NN model definition
INPUT_DIM = X_train.shape[1]

train_dataset = TensorDataset(X_train_input, torch.tensor(le.transform(y_train)))
test_dataset = TensorDataset(X_test_input, torch.tensor(le.transform(y_test)))



{'learning_rate': 1e-05, 'lr_weight_decay': 0.0, 'lr_scheduler': False, 'epochs': 500, 'batch_size': 256, 'drop_sparse_cols': 10, 'dropout': 0.2, 'class_weight_type': 'inverse', 'class_weight_inv_lambda': 10000.0, 'weight_beta': 0.999}
dataset size: (1000, 4097)
After dropping sparse columns: (998, 252)
Abdominal Pain, general                             0.000264
Acute cystitis                                      0.001109
Urolithiasis                                        0.001636
Colitis                                             0.001815
Constipation                                        0.002037
Appendicitis                                        0.002123
Gastritis                                           0.003219
Ovarian cyst                                        0.003441
Bowel obstruction                                   0.003696
Cholangitis                                         0.004158
Flank pain/renal colic                              0.004536
Diverticulitis          

/gpfs/milgram/project/rtaylor/vs428/conda_envs/factcheck_env/lib/python3.7/site-packages/ipykernel_launcher.py:72: FutureWarning: The default value of regex will change from True to False in a future version.


In [80]:
# pd.DataFrame(X_train).describe()
with pd.option_context("display.max_rows", None,"display.max_columns", None):

    display(pd.DataFrame(X_train_input[[1,3,4], :]))
#     display(X_train.iloc[[1,3,4], :])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245
0,tensor(47.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(1.),tensor(1.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(100.),tensor(97.8000),tensor(3.),tensor(64.),tensor(10.),tensor(18.),tensor(116.),tensor(76.)
1,tensor(66.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(0.),tensor(1.),tensor(0.),tensor(0.),tens

In [85]:
with pd.option_context("display.max_rows", None,"display.max_columns", None):
    display(data.loc[[257,376,489], train_col_mask])

,age,DepartmentName_BH EMERGENCY DEPARTMENT,DepartmentName_GH EMERGENCY DEPARTMENT,DepartmentName_SRC EMERGENCY DEPARTMENT,DepartmentName_YNH EMERGENCY ADULT,DepartmentName_YNH EMERGENCY PEDIATRIC,DepartmentName_YNH EMERGENCY SHORELINE,Sex_Female,Sex_Male,GenderIdentity_Female,GenderIdentity_Male,FirstRace_Black or African American,FirstRace_Other,FirstRace_Other/Not Listed,FirstRace_White or Caucasian,Ethnicity_Hispanic or Latino,Ethnicity_Non-Hispanic,PreferredLanguage_English,PreferredLanguage_Spanish,SmokingStatus_Current Every Day Smoker,SmokingStatus_Current Some Day Smoker,SmokingStatus_Former Smoker,SmokingStatus_Never Assessed,SmokingStatus_Never Smoker,SmokingStatus_Never Smoker,AcuityLevel_Emergent,AcuityLevel_Less Urgent,AcuityLevel_Urgent,FinancialClass_BCBS,FinancialClass_Commercial,FinancialClass_Managed Care,FinancialClass_Medicaid,FinancialClass_Medicaid Managed Care,FinancialClass_Medicare,FinancialClass_Medicare Managed Care,FinancialClass_Self-pay,CC_ABDOMINAL PAIN,CC_FLANK PAIN,BLD001,BLD003,BLD007,CIR007,CIR011,CIR012,CIR017,CIR031,DIG004,DIG009,DIG011,DIG012,DIG013,DIG017,DIG019,DIG020,DIG021,DIG022,DIG025,END001,END002,END003,END007,END009,END010,END011,END015,END016,FAC009,FAC014,FAC021,FAC025,GEN002,GEN003,GEN004,GEN005,GEN006,GEN012,INF002,INF007,MBD002,MBD005,MBD024,MUS006,MUS010,MUS011,MUS038,NEO073,NVS010,NVS016,NVS019,RSP002,RSP007,RSP008,RSP009,SKN002,SKN007,SYM002,SYM004,SYM006,SYM007,SYM010,SYM011,SYM012,SYM013,SYM016,SYM017,Analgesic Opioid Agonists,Gastric Acid Secretion Reducing Agents - Proton Pump Inhibitors (PPIs),Antianxiety Agent - Benzodiazepines,Anticonvulsant - GABA Analogs,Glucocorticoids,Antihyperlipidemic - HMG CoA Reductase Inhibitors (statins),Antidepressant - Selective Serotonin Reuptake Inhibitors (SSRIs),NSAID Analgesics (COX Non-Specific) - Propionic Acid Derivatives,Laxative - Saline and Osmotic,ACE Inhibitors,Analgesic or Antipyretic Non-Opioid,Skeletal Muscle Relaxant - Central Muscle Relaxants,Antiemetic - Selective Serotonin 5-HT3 Antagonists,Insulin Analogs - Long Acting,"Asthma/COPD Therapy - Beta 2-Adrenergic Agents, Inhaled, Short Acting",Salicylate Analgesics,Antipsychotic -Atypical Dopamine-Serotonin Antag-Dibenzothiazepine Der,Dermatological - Topical Local Anesthetic Amides,Antidepressant - Serotonin-2 Antagonist-Reuptake Inhibitors (SARIs),Diuretic - Loop,Beta Blockers Cardiac Selective,Calcium Channel Blockers - Dihydropyridines,Antianxiety Agent - Antihistamine Type,Gastric Acid Secretion Reducers - Histamine H2-Receptor Antagonists,Laxative - Surfactant,Thyroid Hormones - Synthetic T4 (Thyroxine),Cephalosporin Antibiotics - 1st Generation,Medical Supplies and DME - Blood Glucose Tests,Alpha-Beta Blockers,Nasal Corticosteroids,Anticoagulants - Coumarin,Insulin Analogs - Rapid Acting,Antibacterial Folate Antagonist - Other Combinations,Analgesic Opioid Hydrocodone Combinations,Antidepressant - Serotonin-Norepinephrine Reuptake Inhibitors (SNRIs),Dermatological - Glucocorticoid,Aminopenicillin Antibiotic - Beta-lactamase Inhibitor Combinations,Tetracycline Antibiotics,Antihistamines - 2nd Generation,Laxative - Stimulant,Minerals and Electrolytes - Iron,Asthma/COPD Therapy - Beta Adrenergic-Glucocorticoid Combinations,Asthma Therapy - Leukotriene Receptor Antagonists,"Minerals and Electrolytes - Potassium, Oral",Antiprotozoal-Antibacterial 1st Generation 2-methyl-5-nitroimidazole,Fluoroquinolone Antibiotics,Antianginal - Coronary Vasodilators (Nitrates),Gastrointestinal Prokinetic Agents - D2 Antagonist/5-HT4 Agonists,Antidepressant - Alpha-2 Receptor Antagonists (NaSSA),Antidepressant-Norepinephrine and Dopamine Reuptake Inhibitors (NDRIs),"Vitamins - B-12, Cyanocobalamin and derivatives",Angiotensin II Receptor Blockers (ARBs),Antidepressant-Tricyclics and Related (Non-Select Reuptake Inhibitors),Analgesic Opioid Oxycodone Combinations,Platelet Aggregation Inhibitors - Thienopyridine Agents,Vitamins - D Derivatives,Asthma Therapy - Inhaled Corticosteroids (Gl

In [86]:
data.columns.get_loc("Antihyperlipidemic - HMG CoA Reductase Inhibitors (statins)")

115

In [76]:
le.classes_, le.transform(y_train)

(array(['Abdominal Pain, general', 'Acute cystitis',
        'Acute gastroenteritis', 'Appendicitis', 'Bowel obstruction',
        'Bowel perforation', 'CHF', 'Cholangitis', 'Colitis',
        'Constipation', 'Diabetic ketoacidosis', 'Diverticulitis',
        'Ectopic pregnancy', 'Flank pain/renal colic', 'Gastritis',
        'Gastroesophageal reflux disease (GERD)', 'Gastroparesis',
        'Hepatitis', 'Hernia', 'Inflammatory bowel disease',
        'Irritable bowel syndrome', 'Mesenteric ischemia',
        'Mesenteric lymphadenitis', 'Methanol Poisoning',
        'Myoma (degenerating)', 'Ovarian cyst', 'Pancreatitis',
        'Pelvic inflammatory disease',
        'Peptic ulcer disease with or without perforation', 'Peritonitis',
        'Pneumonia', 'Pulmonary embolism', 'Renal Failure', 'STI',
        'Sepsis', 'Strep Pharyngitis (peds)', 'Urolithiasis',
        'Vaginitis (e.g. trichomoniasis, candidiasis)[1]',
        'Viral infection/influenza'], dtype=object),
 array([ 0,  0, 

In [77]:
std_scaler.feature_names_in_
# minmax_scaler.feature_names_in_

array(['age', 'last_SpO2', 'last_Temp', 'last_Patient Acuity',
       'last_Pulse', 'last_Pain Score', 'last_Resp', 'last_BP_Systolic',
       'last_BP_Diastolic', 'ed_SpO2', 'ed_Temp', 'ed_Patient Acuity',
       'ed_Pulse', 'ed_Pain Score', 'ed_Resp', 'ed_BP_Systolic',
       'ed_BP_Diastolic'], dtype=object)

['PAT_ENC_CSN_ID',
 'person_id',
 'visit_occurrence_id',
 'BLD001',
 'BLD003',
 'BLD007',
 'CIR007',
 'CIR011',
 'CIR012',
 'CIR017',
 'CIR031',
 'DIG004',
 'DIG009',
 'DIG011',
 'DIG012',
 'DIG013',
 'DIG017',
 'DIG019',
 'DIG020',
 'DIG021',
 'DIG022',
 'DIG025',
 'END001',
 'END002',
 'END003',
 'END007',
 'END009',
 'END010',
 'END011',
 'END015',
 'END016',
 'FAC009',
 'FAC014',
 'FAC021',
 'FAC025',
 'GEN002',
 'GEN003',
 'GEN004',
 'GEN005',
 'GEN006',
 'GEN012',
 'INF002',
 'INF007',
 'MBD002',
 'MBD005',
 'MBD024',
 'MUS006',
 'MUS010',
 'MUS011',
 'MUS038',
 'NEO073',
 'NVS010',
 'NVS016',
 'NVS019',
 'RSP002',
 'RSP007',
 'RSP008',
 'RSP009',
 'SKN002',
 'SKN007',
 'SYM002',
 'SYM004',
 'SYM006',
 'SYM007',
 'SYM010',
 'SYM011',
 'SYM012',
 'SYM013',
 'SYM016',
 'SYM017',
 'Analgesic Opioid Agonists',
 'Gastric Acid Secretion Reducing Agents - Proton Pump Inhibitors (PPIs)',
 'Antianxiety Agent - Benzodiazepines',
 'Anticonvulsant - GABA Analogs',
 'Glucocorticoids',
 'Antih

In [114]:
data[train_col_mask]

,age,DepartmentName_BH EMERGENCY DEPARTMENT,DepartmentName_BH FAIRFIELD URGENT CARE,DepartmentName_GH EMERGENCY DEPARTMENT,DepartmentName_LMH EMERGENCY DEPARTMENT,DepartmentName_LMH EMERGENCY PEQUOT,DepartmentName_MC EMERGENCY DEPARTMENT,DepartmentName_MC URGENT CARE,DepartmentName_SRC EMERGENCY DEPARTMENT,DepartmentName_WH EMERGENCY DEPARTMENT,...,last_BP_Systolic,last_BP_Diastolic,ed_SpO2,ed_Temp,ed_Patient Acuity,ed_Pulse,ed_Pain Score,ed_Resp,ed_BP_Systolic,ed_BP_Diastolic
59090,28.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,100.0,95.9,3.0,87.0,6.0,20.0,101.0,60.0
123317,23.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,119.0,55.0,100.0,98.2,3.0,74.0,7.0,18.0,130.0,84.0
67357,56.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,98.0,98.1,4.0,75.0,4.0,16.0,150.0,93.0
42213,43.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,99.0,61.0,98.0,97.9,3.0,89.0,8.0,20.0,116.0,71.0
47903,71.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,189.0,82.0,100.0,97.8,3.0,69.0,0.0,16.0,195.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180137,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,97.0,98.4,3.0,125.0,0.0,22.0,82.0,52.0
180284,9.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,66.0,99.0,99.0,4.0,115.0,2.0,20.0,94.0,62.0
180892,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,111.0,56.0,98.0,98.3,3.0,85.0,8.0,17.0,108.0,79.0
181272,8.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,101.0,71.0,96.0,98.3,3.0,150.0,0.0,20.0,NaN,NaN
